<font size = "5">Загрузка и предобработка данных

In [1]:
import pandas as pd

In [2]:
train_df = pd.read_csv("train_spam.csv")
test_df = pd.read_csv("test_spam.csv")

In [3]:
train_df.head()

,text_type,text
0,ham,make sure alex knows his birthday is over in f...
1,ham,a resume for john lavorato thanks vince i will...
2,spam,plzz visit my website moviesgodml to get all m...
3,spam,urgent your mobile number has been awarded wit...
4,ham,overview of hr associates analyst project per ...


In [4]:
train_df["text_type"].replace("spam", 1, inplace = True)
train_df["text_type"].replace("ham", 0, inplace = True)

In [5]:
train_df["text_type"].value_counts()

text_type
0    11469
1     4809
Name: count, dtype: int64

В выборке наблюдается небольшой дисбаланс классов.

Рассмотрим распределения текстов по длине.

In [6]:
train_df["text"].apply(lambda x: len(x.split(" "))).describe()

count    16278.000000
mean        56.847094
std         52.170330
min          1.000000
25%         12.000000
50%         30.500000
75%        114.000000
max        206.000000
Name: text, dtype: float64

In [7]:
test_df["text"].apply(lambda x: len(x.split(" "))).describe()

count    4070.00000
mean       57.31769
std        52.41558
min         1.00000
25%        12.00000
50%        30.00000
75%       114.75000
max       176.00000
Name: text, dtype: float64

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [9]:
TfidfVectorizer().fit_transform(pd.concat([train_df["text"], test_df["text"]]).values).shape

(20348, 60317)

Без предобработки в трейне и тесте примерно 60 тыс. терминов.

In [10]:
train_df["text"].values[4]

'overview of hr associates analyst project per david s request attached is an overview of the hr associates analysts project creating a human resource value index this document will provide a brief top line overview of the following description of the challenges people involved positive outcomes high level description of the process we suggest if you have any questions before our tuesday meeting please contact either myself or dan brown thanks tana cashion david oxley ect 10 05 2000 10 20 am to gerry gibson corp enron enron cc andrea yowman corp enron enron bob sparger corp enron enron tim o rourke corp enron enron ted c bland hou ect ect daniel brown na enron enron tana cashion na enron enron rhonna palmer hou ect ect cindy'

В сообщениях со спамом присутствуют эмодзи, проверим насколько часто.

In [11]:
train_df.iloc[2]["text"]

'plzz visit my website moviesgodml to get all movies for free and also i provide direct download links no redirect and ads😊😊😊😊😁'

In [12]:
train_df.iloc[4749]["text"]

'follow my account guys😘❤️'

In [13]:
import regex

In [14]:
temp = train_df["text"][train_df["text_type"] == 1].\
                                    apply(lambda x: len([emoji for emoji in regex.findall(r"\p{Emoji = Yes}", x) 
                                              if ord(emoji) > 100]))
print("Доля сообщений с эмодзи в спаме: ", round(temp[temp != 0].shape[0] / temp.shape[0], 3))

Доля сообщений с эмодзи в спаме:  0.276


In [15]:
#Среднее число эмодзи в спаме:
temp[temp != 0].mean()

9.058867924528302

In [16]:
temp = train_df["text"][train_df["text_type"] == 0].\
                                    apply(lambda x: len([emoji for emoji in regex.findall(r"\p{Emoji = Yes}", x) 
                                              if ord(emoji) > 100]))
print("Доля сообщений с эмодзи не в спаме: ", round(temp[temp != 0].shape[0] / temp.shape[0], 3))

Доля сообщений с эмодзи не в спаме:  0.007


В ~27% спама присутствуют эмодзи, причем несколько в каждом, а не в спаме лишь в 0.7%. Они оказывают значительное влияние на классификацию, заменим их на слово "emoji".

In [17]:
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

В сообщениях изначально были удалены знаки препинания и апострофы,и из-за этого появились несуществующие слова по типу "t", "d". Грамматическая связность текстов не оказывает влияние на классификацию, но увеличивает количество терминов, поэтому служебные части речи можно удалить. Для этой же цели необходимо применить лемматизацию. Также удаляются все слова короче двух символдов и цифры.

In [18]:
lemmatizer = WordNetLemmatizer()
def preprocessMessages(text):
    """
    Функция предобработка сообщений: удаление цифр, стоп-слов, коротких слов, замена эмодзи, а также лемматизация.  
    """
    text = re.sub(r"\d", "", text)
    text = re.sub(r"\s+|\s+".join(stopwords.words("english")), " ", text)
    text = re.sub(r"\s+.{,2}\s+", " ", text)
    text = regex.sub(r"\p{Emoji = Yes}", " emoji ", text)
    lemmatized_tokens = [lemmatizer.lemmatize(word.strip()) for word in text.split(" ")]
    text = " ".join(lemmatized_tokens)
    return text

In [19]:
prep_messages = pd.concat([train_df["text"], test_df["text"]]).apply(preprocessMessages)

In [20]:
TfidfVectorizer().fit_transform(prep_messages).shape

(20348, 50537)

Количество терминов сократилось на 15 %.

In [21]:
prep_messages[:16278].apply(lambda x: len(x.split(" "))).describe()

count    16278.000000
mean        36.327006
std         32.384604
min          1.000000
25%          8.000000
50%         20.000000
75%         71.000000
max        290.000000
Name: text, dtype: float64

In [22]:
prep_messages[16278:].apply(lambda x: len(x.split(" "))).describe()

count    4070.000000
mean       36.487469
std        32.046403
min         1.000000
25%         8.000000
50%        20.000000
75%        71.000000
max       175.000000
Name: text, dtype: float64

После предобработки распределения сообщений по длине слов в трейне и тесте остались схожими.

<font size = "5">Обучение моделей

Сравним метод ближайших соседей и два ансамблевых алгоритма: случайный лес и xgboost. Для каждого из алгоритмов подберем оптимальные гиперпамаметры по значению roc_auc на тренировочный выборке.

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [24]:
def constructFeatures(texts, n):
    """
    Функция векторизации сообщений и разбиения на тренировочную, тестовую и валидационную выборки
    """
    features = TfidfVectorizer(min_df = n).fit(texts)
    features = features.transform(texts)
    print("кол-во терминов tf-idf : ", features.shape[1])
    X_train = features[:16278]
    X_test = features[16278:]
    y_train = train_df["text_type"].values
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 30)
    return X_train, X_val, y_train, y_val, X_test

In [25]:
features = TfidfVectorizer(min_df = 5).fit(prep_messages)

In [33]:
#Пример убранных терминов, часто всречаются редкие наречия, прилагательные, слова с орфографическими ошибками, не из английского языка
list(features.stop_words_)[:100]

['mesg',
 'greentree',
 'exponent',
 'wahleykkumsharing',
 'bault',
 'marx',
 'vless',
 'restaur',
 'mga',
 'cornwall',
 'nerveless',
 '𝕎𝕖𝕝𝕔𝕠𝕞𝕖',
 'sclerosis',
 'drgv',
 '𝙙𝙤𝙣𝙩',
 'ptb',
 'hyunda',
 'repaired',
 'jusqu',
 'taunted',
 'reflection',
 'carpet',
 'handspune',
 'ghajin',
 'thosethings',
 'kanoanyway',
 'healing',
 'powerbank',
 'kbrte',
 '𝙨𝙝𝙖𝙧𝙚',
 'missng',
 '𝐩𝐚𝐫𝐚𝐝𝐞',
 '𝘺𝘰𝘶𝘳',
 'shubham',
 'beerrs',
 'rese',
 'grouporiginal',
 'invulnerability',
 'morton',
 'olsovsky',
 'santana',
 'scrpiting',
 'reasonomatic',
 'ᴍᴜsᴛ',
 'followersrs',
 'mercenary',
 'matrimony',
 'ploy',
 'stre',
 '𝘰𝘷𝘦𝘳',
 'impractical',
 'monograph',
 'stox',
 'efs',
 'sendgrid',
 'singature',
 '𝒑𝒆𝒂𝒄𝒆',
 'currantly',
 '𝘾𝙍𝙀𝘼𝙏𝙊𝙍',
 'marginally',
 '𝗁𝖺𝗌',
 '𝚛𝚎𝚌𝚘𝚖𝚖𝚎𝚗𝚍',
 'vvalk',
 'intermediation',
 'fedora',
 'ғroм',
 'cjw',
 'richter',
 'shortcut',
 'payback',
 'lawrencelrtnmt',
 'istlef',
 'ferrer',
 'infotxtcouk',
 'sumthin',
 '𝙋𝘼𝙎𝙎',
 'cbc',
 'kehna',
 'omnicom',
 'namefather',
 'yesall',
 'fxshawnjsinek',

Сообщения в tf-idf - разряженные векторы, поэтому при уменьшении размерности как правило будут отсекаться нулевые значения. Это не повлияет на качество классификации в, поэтому для скорости обучения лучше уменьшить размерность.

<font size = "4">1. KNN

In [26]:
from sklearn.neighbors import KNeighborsClassifier

In [27]:
from sklearn.model_selection import GridSearchCV

Сравним классификации при помощи перебора гиперпараметров(количества ближайших соседей) для минимальной втречаемости терминов : 5, 20, 50, 100 в tf-ifd.

In [28]:
X_train, X_val, y_train, y_val, X_test = constructFeatures(prep_messages, 5)

кол-во терминов tf-idf :  10093


In [29]:
parameters = {"n_neighbors": [3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25]}
clf = GridSearchCV(KNeighborsClassifier(), param_grid = parameters, scoring = "roc_auc", verbose = 1)
clf.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23,
                                         25]},
             scoring='roc_auc', verbose=1)

In [30]:
print("ROC AUC train: ", clf.best_score_, "\nbest params: ", clf.best_params_, 
      "\nROC AUC val: ",  roc_auc_score(y_val, clf.predict(X_val)))

ROC AUC train:  0.7765725650489304 
best params:  {'n_neighbors': 5} 
ROC AUC val:  0.6806647528552066


In [31]:
X_train, X_val, y_train, y_val, X_test = constructFeatures(prep_messages, 20)

кол-во терминов tf-idf :  3699


In [32]:
parameters = {"n_neighbors": [3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25]}
clf = GridSearchCV(KNeighborsClassifier(), param_grid = parameters, scoring = "roc_auc", verbose = 1)
clf.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23,
                                         25]},
             scoring='roc_auc', verbose=1)

In [33]:
print("ROC AUC train: ", clf.best_score_, "\nbest params: ", clf.best_params_, 
      "\nROC AUC val: ",  roc_auc_score(y_val, clf.predict(X_val)))

ROC AUC train:  0.816146513327829 
best params:  {'n_neighbors': 7} 
ROC AUC val:  0.6743511223603009


In [34]:
X_train, X_val, y_train, y_val, X_test = constructFeatures(prep_messages, 50)

кол-во терминов tf-idf :  1829


In [35]:
parameters = {"n_neighbors": [3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25]}
clf = GridSearchCV(KNeighborsClassifier(), param_grid = parameters, scoring = "roc_auc", verbose = 1)
clf.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23,
                                         25]},
             scoring='roc_auc', verbose=1)

In [36]:
print("ROC AUC train: ", clf.best_score_, "\nbest params: ", clf.best_params_, 
      "\nROC AUC val: ",  roc_auc_score(y_val, clf.predict(X_val)))

ROC AUC train:  0.837449045378772 
best params:  {'n_neighbors': 7} 
ROC AUC val:  0.693894325544492


In [37]:
X_train, X_val, y_train, y_val, X_test = constructFeatures(prep_messages, 100)

кол-во терминов tf-idf :  994


In [38]:
parameters = {"n_neighbors": [3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25]}
clf = GridSearchCV(KNeighborsClassifier(), param_grid = parameters, scoring = "roc_auc", verbose = 1)
clf.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23,
                                         25]},
             scoring='roc_auc', verbose=1)

In [39]:
print("ROC AUC train: ", clf.best_score_, "\nbest params: ", clf.best_params_, 
      "\nROC AUC val: ",  roc_auc_score(y_val, clf.predict(X_val)))

ROC AUC train:  0.8597921586450108 
best params:  {'n_neighbors': 7} 
ROC AUC val:  0.7037699730861773


Практически идентичное значение метрик на тренировочной и валидационной выборках

<font size = "4">2. Random forest

In [40]:
from sklearn.ensemble import RandomForestClassifier

In [44]:
parameters = {"n_jobs": [-1], "random_state": [10] , "n_estimators": [50, 150, 200], "max_depth": [None, 2, 3, 5],
            "min_samples_split": [2, 4, 6], "min_samples_leaf": [1, 2]}

clf = GridSearchCV(RandomForestClassifier(), param_grid = parameters, scoring = "roc_auc", verbose = 1)
clf.fit(X_train, y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'max_depth': [None, 2, 3, 5],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 4, 6],
                         'n_estimators': [50, 150, 200], 'n_jobs': [-1],
                         'random_state': [10]},
             scoring='roc_auc', verbose=1)

In [45]:
print("ROC AUC train: ", clf.best_score_, "\nbest params: ", clf.best_params_, 
      "\nROC AUC val: ",  roc_auc_score(y_val, clf.predict(X_val)))

ROC AUC train:  0.9683644010841537 
best params:  {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 6, 'n_estimators': 200, 'n_jobs': -1, 'random_state': 10} 
ROC AUC val:  0.9091288263209306


<font size = "4">3. XGBOOST

XGBoost обучается дольше, поэтому параметры будем перебирать пошагово.

In [50]:
import xgboost as xgb

In [51]:
import warnings
warnings.filterwarnings("ignore")

In [64]:
parameters = {"n_jobs": [-1], "random_state": [10] , "n_estimators": [100, 200], "max_depth": [5, 7],
             "learning_rate": [1, 1.5],  "alpha": [0.8, 1.2], "reg_lambda": [0.6, 1]}
clf = GridSearchCV(xgb.XGBClassifier(), param_grid = parameters, scoring = "roc_auc", verbose = 1)
clf.fit(X_train, y_train)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_b...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             param_grid={'alpha': [0.8, 1.2], 'learning_rate': [1, 1.5],
                         'max_depth': [5, 7], 'n_estimators': [100, 200],
                         'n_jobs': [-1], 'random_state': [10],
                         'reg_lambda': [0.6, 1]},
             scoring='roc_auc', verbose=1)

In [65]:
print("ROC AUC train: ", clf.best_score_, "\nbest params: ", clf.best_params_, 
      "\nROC AUC val: ",  roc_auc_score(y_val, clf.predict(X_val)))

ROC AUC train:  0.9687998424011971 
best params:  {'alpha': 0.8, 'learning_rate': 1, 'max_depth': 5, 'n_estimators': 200, 'n_jobs': -1, 'random_state': 10, 'reg_lambda': 1} 
ROC AUC val:  0.9005827931774839


In [66]:
parameters = {"n_jobs": [-1], "random_state": [10] , "n_estimators": [200, 300, 350], "max_depth": [3, 5],
             "learning_rate": [1],  "alpha": [0.8], "reg_lambda": [1]}
clf = GridSearchCV(xgb.XGBClassifier(), param_grid = parameters, scoring = "roc_auc", verbose = 1)
clf.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_b...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             param_grid={'alpha': [0.8], 'learning_rate': [1],
                         'max_depth': [3, 5], 'n_estimators': [200, 300, 350],
                         'n_jobs': [-1], 'random_state': [10],
                         'reg_lambda': [1]},
             scoring='roc_auc', verbose=1)

In [67]:
print("ROC AUC train: ", clf.best_score_, "\nbest params: ", clf.best_params_, 
      "\nROC AUC val: ",  roc_auc_score(y_val, clf.predict(X_val)))

ROC AUC train:  0.9694374884814076 
best params:  {'alpha': 0.8, 'learning_rate': 1, 'max_depth': 3, 'n_estimators': 350, 'n_jobs': -1, 'random_state': 10, 'reg_lambda': 1} 
ROC AUC val:  0.9007696003775091


In [68]:
parameters = {"n_jobs": [-1], "random_state": [10] , "n_estimators": [350], "max_depth": [3],
             "learning_rate": [1, 1.25],  "alpha": [1, 1.5, 2], "reg_lambda": [1.5, 2]}
clf = GridSearchCV(xgb.XGBClassifier(), param_grid = parameters, scoring = "roc_auc", verbose = 1)
clf.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_b...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             param_grid={'alpha': [1, 1.5, 2], 'learning_rate': [1, 1.25],
                         'max_depth': [3], 'n_estimators': [350],
                         'n_jobs': [-1], 'random_state': [10],
                         'reg_lambda': [1.5, 2]},
             scoring='roc_auc', verbose=1)

In [69]:
print("ROC AUC train: ", clf.best_score_, "\nbest params: ", clf.best_params_, 
      "\nROC AUC val: ",  roc_auc_score(y_val, clf.predict(X_val)))

ROC AUC train:  0.9703562639234595 
best params:  {'alpha': 1, 'learning_rate': 1, 'max_depth': 3, 'n_estimators': 350, 'n_jobs': -1, 'random_state': 10, 'reg_lambda': 1.5} 
ROC AUC val:  0.8995337292287376


In [70]:
parameters = {"n_jobs": [-1], "random_state": [10] , "n_estimators": [300], "max_depth": [1, 3],
             "learning_rate": [1],  "alpha": [1, 1.25], "reg_lambda": [1.5]}
clf = GridSearchCV(xgb.XGBClassifier(), param_grid = parameters, scoring = "roc_auc", verbose = 1)
clf.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_b...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             param_grid={'alpha': [1, 1.25], 'learning_rate': [1],
                         'max_depth': [1, 3], 'n_estimators': [300],
                         'n_jobs': [-1], 'random_state': [10],
                         'reg_lambda': [1.5]},
             scoring='roc_auc', verbose=1)

In [71]:
print("ROC AUC train: ", clf.best_score_, "\nbest params: ", clf.best_params_, 
      "\nROC AUC val: ",  roc_auc_score(y_val, clf.predict(X_val)))

ROC AUC train:  0.970379509456906 
best params:  {'alpha': 1, 'learning_rate': 1, 'max_depth': 3, 'n_estimators': 300, 'n_jobs': -1, 'random_state': 10, 'reg_lambda': 1.5} 
ROC AUC val:  0.8985784085294583


При переборе параметров практически перестали изменяться значения метрик, модель все сильнее переобучается. Выберем наилучшую модель по предсказанию на валидационной выборке и переберем скорость обучения модели.

In [80]:
parameters = {"n_jobs": [-1], "random_state": [10] , "n_estimators": [350], "max_depth": [3],
             "learning_rate": [0.75, 0.8, 0.85, 0.9, 0.95, 1],  "alpha": [0.8], "reg_lambda": [1]}
best_xgb = GridSearchCV(xgb.XGBClassifier(), param_grid = parameters, scoring = "roc_auc", verbose = 1)
best_xgb.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_b...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             param_grid={'alpha': [0.8],
                         'learning_rate': [0.75, 0.8, 0.85, 0.9, 0.95, 1],
                         'max_depth': [3], 'n_estimators': [350],
                         'n_jobs': [-1], 'random_state': [10],
                         'reg_lambda': [1]},
             scoring='roc_auc', verbose=1)

In [82]:
print("ROC AUC train: ", best_xgb.best_score_, "\nbest params: ", best_xgb.best_params_, 
      "\nROC AUC val: ",  roc_auc_score(y_val, best_xgb.predict(X_val)))

ROC AUC train:  0.9720491842546727 
best params:  {'alpha': 0.8, 'learning_rate': 0.75, 'max_depth': 3, 'n_estimators': 350, 'n_jobs': -1, 'random_state': 10, 'reg_lambda': 1} 
ROC AUC val:  0.9058992128737107


In [94]:
best_rf = RandomForestClassifier(max_depth = None, min_samples_leaf = 1, min_samples_split = 6, n_estimators = 200, 
                    n_jobs = -1, random_state = 10).fit(X_train, y_train)

Сравним качество предсказания лучших моделей по каждому из классов.

In [88]:
from sklearn.metrics import confusion_matrix

In [89]:
#тренировочная выборка
confusion_matrix(y_train, best_xgb.predict(X_train))

array([[9110,   66],
       [ 247, 3599]], dtype=int64)

In [86]:
#валидационная выборка
confusion_matrix(y_val, best_xgb.predict(X_val))

array([[2221,   72],
       [ 151,  812]], dtype=int64)

In [90]:
#тренировочная выборка
confusion_matrix(y_train, best_rf.predict(X_train))

array([[9139,   37],
       [ 129, 3717]], dtype=int64)

In [87]:
#валидационная выборка
confusion_matrix(y_val, best_rf.predict(X_val))

array([[2212,   81],
       [ 141,  822]], dtype=int64)

<font size = "5">Выводы

Ансамблевые алгоритмы справились с классификацией значительно лучше KNN. Для классификации тестовых сообщений лучше подойдет случайный лес, так как предсказательная способность на валидационной выборке у него практически совпадает с XGBoost, но обучается он быстрее, что будет критично при увеличении количества сообщений в трейне.

In [95]:
test_df["score"] = best_rf.predict(X_test)

In [100]:
test_df.to_csv("prediction.csv", index = False, sep = ",")